**Note:** All the codes below are run in Saturn notebook. Therefore huge dataset of images are not in github repo. 

**1. check quickly that tensorflow is there in Saturn CLoud**

In [1]:
import tensorflow as tf

2022-11-23 19:00:14.474335: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
tf.__version__

'2.9.1'

## Homework 08

### Dataset

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from Kaggle.

You can get a wget-able version here:

`wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip`

`unzip dino-dragon.zip`

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.

**Note:** You will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). You can also use a computer without a GPU (e.g. your laptop), but it will be slower.

### Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons.

The dataset contains separate folders for training and test sets.

## 2. Getting the images for homework

In [3]:
#uncomment below if data is not downloaded yet
#!wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip

In [5]:
# uncomment below line if data is not unzipped yet
#!unzip dino-dragon.zip

# Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:


* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).


In [6]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

## Developing the model

In [8]:
#https://www.tensorflow.org/guide/keras/save_and_serialize

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',\
             optimizer=optimizers.SGD(learning_rate=0.002, momentum=0.8),\
             metrics=['acc'])

2022-11-23 19:01:37.278799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-23 19:01:37.285212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-23 19:01:37.285843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-23 19:01:37.286960: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

**Answer to question 1**

`binray crossentropy`

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

**Answer to question 2**

`11215873`

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

In [12]:
ls

dino-dragon.zip                       README.md                    test/
hw08-mlzoomcamp-dino-or-dragon.ipynb  single-gpu-tensorflow.ipynb  train/


In [13]:
train_dir = './train'
test_dir = './test'

In [14]:
train_ds = train_gen.flow_from_directory(train_dir,
                                        target_size=(150, 150),
                                        batch_size=20,
                                        class_mode='binary',
                                        shuffle = True)

Found 1594 images belonging to 2 classes.


In [15]:
test_ds = test_gen.flow_from_directory(test_dir,
                                       target_size=(150, 150),
                                       batch_size=20,
                                       class_mode='binary',
                                       shuffle = True)

Found 394 images belonging to 2 classes.


In [16]:
#training
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
)

Epoch 1/10


2022-11-23 19:02:56.738625: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-23 19:02:57.361335: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-23 19:02:57.362267: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-23 19:02:57.362306: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-23 19:02:57.363335: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-23 19:02:57.363428: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


80/80 [==============================] - 12s 122ms/step - loss: 0.6937 - acc: 0.5358 - val_loss: 0.6790 - val_acc: 0.4975
Epoch 2/10
80/80 [==============================] - 8s 105ms/step - loss: 0.6495 - acc: 0.6587 - val_loss: 0.6297 - val_acc: 0.5711
Epoch 3/10
80/80 [==============================] - 8s 106ms/step - loss: 0.5588 - acc: 0.7547 - val_loss: 0.5032 - val_acc: 0.8020
Epoch 4/10
80/80 [==============================] - 8s 106ms/step - loss: 0.4654 - acc: 0.7936 - val_loss: 0.4115 - val_acc: 0.8477
Epoch 5/10
80/80 [==============================] - 8s 106ms/step - loss: 0.4154 - acc: 0.8231 - val_loss: 0.3802 - val_acc: 0.8579
Epoch 6/10
80/80 [==============================] - 9s 106ms/step - loss: 0.3597 - acc: 0.8513 - val_loss: 0.3403 - val_acc: 0.8528
Epoch 7/10
80/80 [==============================] - 8s 105ms/step - loss: 0.3218 - acc: 0.8720 - val_loss: 0.3590 - val_acc: 0.8426
Epoch 8/10
80/80 [==============================] - 8s 105ms/step - loss: 0.2951 - acc

In [17]:
acc = history.history['acc']
acc_median = np.median(acc)
acc_median

0.8372020125389099

In [18]:
loss = history.history['loss']
loss_std = np.std(loss)
loss_std

0.15246697522258218

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

**Answer to question 3**

`0.90` (closest answer to 0.86)

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

**Answer to question 4**
`0.11` (closest answer to my answer 0.15)

## Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [19]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

In [21]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [22]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150), 
                                                    batch_size=32, 
                                                    class_mode='binary')

Found 1594 images belonging to 2 classes.


In [23]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 394 images belonging to 2 classes.


In [24]:
# training
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
50/50 [==============================] - 15s 295ms/step - loss: 0.4631 - acc: 0.7886 - val_loss: 0.3560 - val_acc: 0.8426
Epoch 2/10
50/50 [==============================] - 15s 292ms/step - loss: 0.4458 - acc: 0.7986 - val_loss: 0.2937 - val_acc: 0.8756
Epoch 3/10
50/50 [==============================] - 15s 292ms/step - loss: 0.4576 - acc: 0.7898 - val_loss: 0.3034 - val_acc: 0.8756
Epoch 4/10
50/50 [==============================] - 14s 290ms/step - loss: 0.4213 - acc: 0.8087 - val_loss: 0.3263 - val_acc: 0.8629
Epoch 5/10
50/50 [==============================] - 14s 289ms/step - loss: 0.4234 - acc: 0.8137 - val_loss: 0.3716 - val_acc: 0.8350
Epoch 6/10
50/50 [==============================] - 14s 289ms/step - loss: 0.3995 - acc: 0.8187 - val_loss: 0.2795 - val_acc: 0.8731
Epoch 7/10
50/50 [==============================] - 14s 290ms/step - loss: 0.3923 - acc: 0.8174 - val_loss: 0.3651 - val_acc: 0.8426
Epoch 8/10
50/50 [==============================] - 14s 289ms/step - 

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [25]:
val_loss_aug = history.history['val_loss']
loss_mean_aug = np.mean(val_loss_aug)
loss_mean_aug

0.3693745702505112

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [26]:
val_acc_aug = history.history['val_acc']
acc_mean_aug = np.mean(val_acc_aug[5:10])
acc_mean_aug

0.8167512655258179

## Submit the results

- Submit your results here: https://forms.gle/XdH5ztBddvTvxzpT6
- You can submit your solution multiple times. In this case, only the last submission will be used
- If your answer doesn't match options exactly, select the closest one


## Deadline

The deadline for submitting is 21 November 2022, 23:00 CEST.

After that, the form will be closed.